# Plugin loading

In [4]:
# Install and load Cuda plugin for Google colab
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0syocffq
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0syocffq
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmphidb9bgs".


In [9]:
!nvidia-smi

Thu Dec 12 21:47:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Code

In [16]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>
#include <assert.h>

#define MATRIX_SIZE 8192
#define MASK_SIZE 3
#define CHECK_MALLOC(ptr) {if(ptr == NULL) {printf("Not enough memory!");exit(1);}}

__constant__ int mask_device[MASK_SIZE*MASK_SIZE];

void
cudaInfo() {
    // retrieve some info about the CUDA device
    int nDevices;
    cudaGetDeviceCount(&nDevices);
    for (int i = 0; i < nDevices; i++) {
      cudaDeviceProp prop;
      cudaGetDeviceProperties(&prop, i);
      printf("Device Number: %d\n", i);
      printf("  Device name: %s\n", prop.name);
      printf("  max Blocks Per MultiProcessor: %d\n", prop.maxBlocksPerMultiProcessor);
      printf("  max Threads Per MultiProcessor: %d\n", prop.maxThreadsPerMultiProcessor);
      printf("  max Threads Per Block: %d\n", prop.maxThreadsPerBlock);
      printf("  num SM: %d\n", prop.multiProcessorCount);
      printf("  num bytes sharedMem Per Block: %d\n", prop.sharedMemPerBlock);
      printf("  num bytes sharedMem Per Multiprocessor: %d\n", prop.sharedMemPerMultiprocessor);
      printf("  Memory Clock Rate (KHz): %d\n",
           prop.memoryClockRate);
      printf("  Memory Bus Width (bits): %d\n",
           prop.memoryBusWidth);
      printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
           2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
    }
}

bool
areEqual(int* mat1, int* mat2, int mat_dim) {
    for(int i = 0; i < mat_dim; ++i) {
        for(int j = 0; j < mat_dim; ++j) {
            if(mat1[j+mat_dim*i] != mat2[j+mat_dim*i]) {
                printf("[INFO] Test failed at (%d,%d)", i, j);
                return false;
            }
        }
    }
    return true;
}

void
initMatrix(
    int* mat,
    int mat_dim
) {
    for(int i = 0; i < mat_dim; ++i) {
        mat[i] = 0;
    }
}

void
convCpu(
    int* mat_in,
    int mat_dim,
    int* mask,
    int mask_dim,
    int* mat_out
) {
    int half_mask = mask_dim/2;
    for(int i = 0; i < mat_dim; ++i) {
        for(int j = 0; j < mat_dim; ++j) {
            mat_out[j + i*mat_dim] = 0;
            for(int k = -half_mask; k <= half_mask; ++k) {
                for(int l = -half_mask; l <= half_mask; ++l) {
                    if(j+l >= 0 && j+l < mat_dim && i+k >= 0 && i+k < mat_dim) {
                        mat_out[j + i*mat_dim] +=
                            mat_in[(j+l) + (i+k)*mat_dim] *
                            mask[(half_mask+l) + (half_mask+k)*mask_dim];
                    }
                }
            }
        }
    }
}

__global__ void
convKernel2D(
    int* mat_in,
    int mat_dim,
    int mask_dim,
    int* mat_out
) {
    int row = threadIdx.y + blockDim.y * blockIdx.y;
    int col = threadIdx.x + blockDim.x * blockIdx.x;

    // Boundaries check (We might have more threads than strictly necessary)
    if(row < mat_dim && col < mat_dim) {
        int half_mask = mask_dim/2;
        int tmp_sum = 0;
        for(int i = -half_mask; i <= half_mask; ++i) {
            for(int j = -half_mask; j <= half_mask; ++j) {
                if(col+j >= 0 && col+j < mat_dim && row+i >= 0 && row+i < mat_dim) {
                    tmp_sum +=
                        mat_in[(col+j) + (row+i)*mat_dim] *
                        mask_device[(half_mask+j)+(half_mask+i)*mask_dim];
                }
            }
        }
        mat_out[col + row*mat_dim] = tmp_sum;
    }

}

__global__ void
convKernel2DTiling(
    int* mat_in,
    int mat_dim,
    int mask_dim,
    int* mat_out
) {
    // All the threads collaborate to load a tile into the shared memory
    extern __shared__ int tile[];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int offset = (mask_dim-1)/2;
    // int col_i = blockIdx.x*blockDim.x - offset + tx;
    // int row_i = blockIdx.y*blockDim.y - offset + ty;
    int col_i = blockIdx.x*(blockDim.x-2*offset) - offset + tx;
    int row_i = blockIdx.y*(blockDim.y-2*offset) - offset + ty;

    if(row_i >= 0 && row_i < mat_dim && col_i >= 0 && col_i < mat_dim) {
        tile[tx + blockDim.x * ty] = mat_in[col_i + mat_dim * row_i];
    } else {
        tile[tx + blockDim.x * ty] = 0;
    }
    __syncthreads();

    // Only certain threads are going to produce an output
    if(tx >= offset && ty >= offset && tx < (blockDim.x-offset) && ty < (blockDim.y-offset)) {
        int half_mask = mask_dim/2;
        int tmp_sum = 0;
        for(int i = -half_mask; i <= half_mask; ++i) {
            for(int j = -half_mask; j <= half_mask; ++j) {
                tmp_sum +=
                    tile[(tx+j) + (ty+i)*blockDim.x] *
                    mask_device[(half_mask+j)+(half_mask+i)*mask_dim];
            }
        }
        // Boundary check for output writing
        if(col_i < mat_dim && row_i < mat_dim) {
            mat_out[col_i + mat_dim * row_i] = tmp_sum;
        }
    }
}


int main(int argc, char const *argv[])
{
    cudaInfo();

    int matrix_num_el = MATRIX_SIZE*MATRIX_SIZE;
    int mask_num_el = MASK_SIZE*MASK_SIZE;

    // Allocate host memory and initialize it
    int* mat_in = (int*)malloc(matrix_num_el*sizeof(int));
    CHECK_MALLOC(mat_in);
    int* mat_out = (int*)malloc(matrix_num_el*sizeof(int));
    CHECK_MALLOC(mat_out);
    int* mask = (int*)malloc(mask_num_el*sizeof(int));
    CHECK_MALLOC(mask);

    for(int i = 0; i < matrix_num_el; ++i) {
        mat_in[i] = 2;
    }

    for(int i = 0; i < mask_num_el; ++i) {
        mask[i] = 10;
    }

    // ************ CPU COMPUTATION **************
    int* mat_out_tmp = (int*)malloc(matrix_num_el*sizeof(int));
    CHECK_MALLOC(mat_out_tmp);

    clock_t begin = clock();
    convCpu(mat_in, MATRIX_SIZE, mask, MASK_SIZE, mat_out_tmp);
    clock_t end = clock();

    double time_spent = ((double)((end - begin)) * 1000) / CLOCKS_PER_SEC;
    printf("\n\n[CPU] matrix_size: %dx%d\n", MATRIX_SIZE, MATRIX_SIZE);
    printf("  - CPU time: %f ms\n", time_spent);
    // *******************************************



    // ************ GPU COMPUTATION **************
    cudaError_t error;

    // Allocate device memory and initialize it
    int* mat_in_device;
    cudaMalloc(&mat_in_device, matrix_num_el*sizeof(int));
    cudaMemcpy(mat_in_device, mat_in, matrix_num_el*sizeof(int), cudaMemcpyHostToDevice);

    int* mat_out_device;
    cudaMalloc(&mat_out_device, matrix_num_el*sizeof(int));

    cudaMemcpyToSymbol(mask_device, mask, mask_num_el*sizeof(int));

    printf("\n\n[GPU] matrix_size: %dx%d\n", MATRIX_SIZE, MATRIX_SIZE);
    // Test for different block sizes
    for(int i = 4; i <= 32; i*=2) {
        printf("[GPU] block_size: %d\n", i);
        // Kernel parameters
        int thread_num_x = i;
        int thread_num_y = thread_num_x;

        int block_num_x = (MATRIX_SIZE) / thread_num_x;
        int block_num_y = (MATRIX_SIZE) / thread_num_y;

        dim3 grid_dim(block_num_x,block_num_y);
        dim3 block_dim(thread_num_x, thread_num_y);

        // In the tiled algorithm the number of threads per block is increased
        //  by a certain number to account for the halo
        dim3 grid_dim_tiling(block_num_x, block_num_y);
        int thread_num_x_tiling = thread_num_x+MASK_SIZE-1;
        int thread_num_y_tiling = thread_num_y+MASK_SIZE-1;
        dim3 block_dim_tiling(thread_num_x_tiling,thread_num_y_tiling);

        // Performance measurment objects
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start, 0);
        convKernel2D<<<grid_dim, block_dim>>>
         (mat_in_device, MATRIX_SIZE, MASK_SIZE, mat_out_device);
        /*
        convKernel2DTiling<<<grid_dim_tiling,
                             block_dim_tiling,
                             thread_num_x_tiling*thread_num_y_tiling*sizeof(int)>>>
                             (mat_in_device, MATRIX_SIZE, MASK_SIZE, mat_out_device);
        */
        cudaDeviceSynchronize();
        error = cudaGetLastError();
        if(error != cudaSuccess) {
            printf("[INFO] Couldn't launch kernel");
            exit(1);
        }
        cudaEventRecord(stop, 0);
        cudaEventSynchronize(stop);

        float gpu_elapsed_time_ms = 0.0;
        cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
        printf("  - GPU time: %f ms\n",
               gpu_elapsed_time_ms, i);

        // Copy back data to device
        cudaMemcpy(mat_out, mat_out_device, matrix_num_el*sizeof(int), cudaMemcpyDeviceToHost);

        // Check results
        if(areEqual(mat_out_tmp, mat_out, MATRIX_SIZE)) {
            printf("  - Test passed\n", i);
        } else {
            printf("  - Test failed\n", i);
        }

        initMatrix(mat_out, matrix_num_el);
    }
    // *******************************************

    int index = (0)+(88)*MATRIX_SIZE;
    printf("mat_out[%d] = %d", index, mat_out[index]);

    free(mat_in);
    free(mat_out);
    free(mat_out_tmp);
    free(mask);

    cudaFree(mat_in_device);
    cudaFree(mat_out_device);
    cudaFree(mask_device);

    return 0;
}

Device Number: 0
  Device name: Tesla T4
  max Blocks Per MultiProcessor: 16
  max Threads Per MultiProcessor: 1024
  max Threads Per Block: 1024
  num SM: 40
  num bytes sharedMem Per Block: 49152
  num bytes sharedMem Per Multiprocessor: 65536
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064000



[CPU] matrix_size: 8192x8192
  - CPU time: 6129.021000 ms


[GPU] matrix_size: 8192x8192
[GPU] block_size: 4
  - GPU time: 23.554079 ms
  - Test passed
[GPU] block_size: 8
  - GPU time: 8.770016 ms
  - Test passed
[GPU] block_size: 16
  - GPU time: 7.171264 ms
  - Test passed
[GPU] block_size: 32
  - GPU time: 8.040768 ms
  - Test passed
mat_out[720896] = 0


# Scratch

In [ ]:
// Matrix multiply example
for(block_size= 4; block_size <= 32; block_size *= 2)
{
    int *a, *b, *c;
    cudaMallocManaged((void **) &a, sizeof(int)*MATRIX_SIZE*MATRIX_SIZE);
    cudaMallocManaged((void **) &b, sizeof(int)*MATRIX_SIZE*MATRIX_SIZE);
    cudaMallocManaged((void **) &c, sizeof(int)*MATRIX_SIZE*MATRIX_SIZE);

    // initialize matrix A
    for (int i = 0; i < MATRIX_SIZE; ++i) {
        for (int j = 0; j < MATRIX_SIZE; ++j) {
            a[i * MATRIX_SIZE + j] = 2;
        }
    }

    // initialize matrix B
    for (int i = 0; i < MATRIX_SIZE; ++i) {
        for (int j = 0; j < MATRIX_SIZE; ++j) {
            b[i * MATRIX_SIZE + j] = 3;
        }
    }


    float  naive_gpu_elapsed_time_ms;

    // some events to count the execution time
    //clock_t st, end;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);


    unsigned int grid_rows = (MATRIX_SIZE + block_size - 1) / block_size;
    unsigned int grid_cols = (MATRIX_SIZE + block_size - 1) / block_size;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(block_size, block_size);


    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(a, b, c, MATRIX_SIZE);
    cudaThreadSynchronize();

    // time counting terminate

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // compute time elapsed on GPU computing
    cudaEventElapsedTime(&naive_gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on naive GPU matrix multiplication of %dx%d . %dx%d (%d): %f ms.\n\n", MATRIX_SIZE, MATRIX_SIZE, MATRIX_SIZE, MATRIX_SIZE, block_size, naive_gpu_elapsed_time_ms);


    // free memory
    cudaFree(a);
    cudaFree(b);
    cudaFree(c);
}